In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [1]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os
MODEL = "gpt-4-0125-preview"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "###################################################"))

def analyze_and_categorize_responses(input_text, theme):
    """
    Analyzes text and categorizes responses into solutions, metrics, and barriers.
    """
    prompt_analysis = (
        f"I want you to give me a bullet list of all the significantly different answers mentioned in a piece of text. \n"
        "Do NOT mention an answer more than once - be sure to remove duplicate points.\n"
        "I am interested in the following types of answers:\n"
        f"- SOLUTION: a way to achieve success in {theme}.\n"
        f"- METRIC: a way to measure the success of {theme}.\n"
        f"- BARRIER: a barrier to the success of {theme}.\n\n"
        "For each point, give me the type of point (METRIC, BARRIER, or SOLUTION) followed by, IN THE SAME LINE, all the text describing that point.\n"
        "Include enough context so each point makes sense on its own.\n"
        "If there are no points in the text, answer NONE.\n\n"
        f"Text: \"{input_text}\""
    )




    analysis_result = client.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":  prompt_analysis},
    ],
        temperature=0.5,
        max_tokens=1500,
        top_p=1.0,
        frequency_penalty=0,
        presence_penalty=0
    ).choices[0].message.content

    return analysis_result

def find_cluster_for_response(question, point, clusters):
    """
    Determines the best cluster for each categorized response.
    """
    prompt_cluster = (
        f"Give me the ID of the best category to put the answer \"{point}\" to the question \"{question}\":\n"
        + "\n".join([f"{cluster_id}: {description}" for cluster_id, description in clusters.items()])
        + "\n\nAnswer in one word with the ID."
    )


    best_cluster_id = client.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": "You are a helpful assistant that can figure out how to categorize text properly."},
        {"role": "user", "content":  prompt_cluster},
    ],
        temperature=0.5,
        max_tokens=1500,
        top_p=1.0,
        frequency_penalty=0,
        presence_penalty=0
    ).choices[0].message.content

    return best_cluster_id

# Example usage
theme = "research"
input_text = '''
              One way I think we can achieve success is by Funding all projects,providing infrastructure,and hiring good people.
             Another way , i think we can measure our success is to measure the output of the research conducted. Good people do critical research.
             The problems I believe we face are Limited Funding: Insufficient financial resources can restrict the scope of research,
              limit access to necessary materials or equipment, and constrain the ability to hire skilled personnel.
            Access to Resources: Difficulty in accessing necessary research materials, data, or equipment can significantly hinder the research process.
            Time Constraints: Researchers often face tight deadlines for completing their work, which can impact the depth and thoroughness of the research.
            Ethical Considerations: Navigating ethical issues, especially in fields like medicine, psychology, and social sciences,
            can be challenging and may limit the types of research that can be conducted.
             '''
clusters = {
    "A": "Improve management",
    "B": "Foster tech Transfer",
    "C": "Support publishing",
    "D": "Personal Qualities",
    "E": "Improve Infrastruture",
    "F": "Smart Topic Selection",
    "G": "Improve Research Funding",
    "H": "Improve Human Capital",
    "I": "Enhance Collaboration",
    "G":"Promote Autonomy"
}

analysis_result = analyze_and_categorize_responses(input_text, theme)
# Assume `analysis_result` is a string with categorized responses.
# You would parse this string to extract individual responses,
# then loop through them to assign each to a cluster.

# This is a simplified example. You'll need to implement parsing of `analysis_result`
# based on its structure, then loop through each response to find its cluster.
for response in analysis_result.split('\n'):  # Simplified; actual parsing depends on analysis_result format.
    best_cluster_id = find_cluster_for_response(theme, response, clusters)
    print(f"Response: {response}\nBest Cluster: {best_cluster_id}\n")


ModuleNotFoundError: No module named 'openai'

In [ ]:
def generate_prompt(answer, category, theme, text):
    # Mapping for argument types to corresponding phrases
    argtype_phrases = {"pro": "is", "con": "is NOT"}

    # Mapping category to corresponding phrase
    category_phrases = {
        "metric": f"a good way to measure success in {theme}",
        "barrier": f"an important barrier to success in {theme}",
        "solution": f"a good way to achieve success in {theme}"
    }

    prompts = []

    for argtype in ["pro", "con"]:
        # Constructing the prompt based on the current argtype
        prompt = (f"Give a bullet list of all the reasons, mentioned in the text below, "
                  f"about why \"{answer}\" {argtype_phrases[argtype]} {category_phrases[category]}.\n"
                  f"\nIf there are no reasons mentioned in the text, answer NONE."
                  f"\n\nGot it? OK. Give me a bullet list of the reasons mentioned in the following text:\n\n\"{text}\"")
        prompts.append(prompt)

    return prompts

# Example usage
answer = "Using AI in education"
category = "solution"
theme = "modern teaching methods"
text = "The integration of AI into education can personalize learning, making it more effective."

prompts = generate_prompt(answer, category, theme, text)

# Print each prompt
for prompt in prompts:
    print(prompt)
    print("\n---\n")


Give a bullet list of all the reasons, mentioned in the text below, about why "Using AI in education" is a good way to achieve success in modern teaching methods.

If there are no reasons mentioned in the text, answer NONE.

Got it? OK. Give me a bullet list of the reasons mentioned in the following text:

"The integration of AI into education can personalize learning, making it more effective."

---

Give a bullet list of all the reasons, mentioned in the text below, about why "Using AI in education" is NOT a good way to achieve success in modern teaching methods.

If there are no reasons mentioned in the text, answer NONE.

Got it? OK. Give me a bullet list of the reasons mentioned in the following text:

"The integration of AI into education can personalize learning, making it more effective."

---



In [ ]:
def generate_prompt(answer, category, theme, text):
    # Mapping for argument types to corresponding phrases
    argtype_phrases = {"pro": "is", "con": "is NOT"}

    # Mapping category to corresponding phrase
    category_phrases = {
        "metric": f"a good way to measure success in {theme}",
        "barrier": f"an important barrier to success in {theme}",
        "solution": f"a good way to achieve success in {theme}"
    }

    prompts = []

    for argtype in ["pro", "con"]:
        # Constructing the prompt based on the current argtype
        prompt = (f"Give a bullet list of all the reasons, mentioned in the text below, "
                  f"about why \"{answer}\" {argtype_phrases[argtype]} {category_phrases[category]}.\n"
                  f"\nIf there are no reasons mentioned in the text, answer NONE."
                  f"\n\nGot it? OK. Give me a bullet list of the reasons mentioned in the following text:\n\n\"{text}\"")
        prompts.append(prompt)

    return prompts

# Example usage
answer = "Using AI in education"
category = "solution"
theme = "modern teaching methods"
text = "The integration of AI into education can personalize learning, making it more effective."

prompts = generate_prompt(answer, category, theme, text)

# Print each prompt
for prompt in prompts:
    print(prompt)
    print("\n---\n")


In [ ]:
def generate_argument_prompts(answer, theme, text):
    prompts = {
        "pro": f"List all the reasons that support the statement: '{answer}' is a good way to achieve success in {theme}.",
        "con": f"List all the reasons against the statement: '{answer}' is not a good way to achieve success in {theme}."
    }
    results = {
        "pro": ["Reason 1 supporting the statement", "Reason 2 supporting the statement"],
        "con": ["Reason 1 against the statement", "Reason 2 against the statement"]
    }
    # For demonstration purposes, we're returning mock results.
    # Replace with actual logic to extract reasons from text.
    return results

def classify_arguments(reasons, criteria):
    # The criteria provided
    criteria_prompts = {
        "actionable": "Is this answer actionable?",
        "high_impact": "Is this answer of high impact?"
    }

    classifications = []

    # Classify each reason into one of the criteria
    for reason in reasons:
        for criterion_key, criterion_question in criteria_prompts.items():
            prompt = (f"Classify the reason '{reason}' into one of these categories:\n"
                      f"1. Actionable\n"
                      f"2. High Impact\n"
                      f"The reason implies that '{reason}' is an answer to the question '{criterion_question}'\n"
                      f"Answer with the number corresponding to the category.")
            classifications.append(prompt)

    return classifications

# Example usage
answer = "Using AI in education"
theme = "modern teaching methods"
text = "The integration of AI into education can personalize learning, making it more effective."
criteria = ["actionable", "high_impact"]

# Generate arguments from the text
arguments = generate_argument_prompts(answer, theme, text)

# Now classify each pro argument
pro_classifications = classify_arguments(arguments['pro'], criteria)
print("Pro Arguments Classification:")
for classification in pro_classifications:
    print(classification)
    print("\n---\n")

# Now classify each con argument
con_classifications = classify_arguments(arguments['con'], criteria)
print("Con Arguments Classification:")
for classification in con_classifications:
    print(classification)
    print("\n---\n")


2ND TRIAL

In [ ]:
import openai

# Set up your API key from an environment variable or directly inputting it
openai.api_key = 'your-api-key'

def extract_points_from_text(theme, text):
    """
    Generates prompts based on a theme and sends them to the OpenAI API
    to extract different answers from a piece of text.
    """
    # List of categories you are interested in
    categories = ["SOLUTION", "METRIC", "BARRIER"]

    # Initialize a list to hold all the prompts
    prompts = []

    # Create prompts for each category
    for category in categories:
        prompt = (
            f"I want you to give me a bullet list of all the significantly different answers mentioned in the text below.\n"
            "Do NOT mention an answer more than once - be sure to remove duplicate points.\n"
            f"I am interested in the following types of answers:\n"
            f"- {category}: a way to {('achieve' if category == 'SOLUTION' else 'measure' if category == 'METRIC' else 'address a barrier to')} success in {theme}.\n"
            "For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.\n"
            "Include enough context so each point makes sense on its own.\n"
            "If there are no points in the text, answer NONE.\n\n"
            f"Text: \"{text}\""
        )
        prompts.append(prompt)

    # Dictionary to store the responses for each category
    points = {category: None for category in categories}

    # Send each prompt to the OpenAI API and store the results
    for prompt, category in zip(prompts, categories):
        response = openai.Completion.create(
            engine="davinci",
            prompt=prompt,
            max_tokens=1024,
            n=1,
            stop=None,
            temperature=0.3
        )
        points[category] = response.choices[0].text.strip()

    return points

# Example usage:
theme = "research"
text = "Here is your text containing different people's responses on how to achieve success in research."
extracted_points = extract_points_from_text(theme, text)

# Print out the categorized points
for category, points in extracted_points.items():
    print(f"{category} Points:")
    print(points)
    print("\n---\n")
